In [1]:
# Setting up google drive 
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [2]:
import models as models
import five_fold_training
import torch
from torch import nn
import torch.utils.data as data_utils
from torch.utils.data import DataLoader, ConcatDataset
from IPython import display
import math
import torchvision
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
train_path = '/content/drive/MyDrive/Data/MM_train_data'
train_data = torch.load(train_path)
test_path = '/content/drive/MyDrive/Data/MM_test_data'
test_data = torch.load(test_path)


In [4]:
dataset = ConcatDataset([test_data, train_data])

In [5]:
net50 = models.AE2(input_size=204, 
      hidden_size1=196, 
      hidden_size2=128, 
      latent_size=50)

In [6]:
net15b2 = models.BernoulliVAE2(input_size=204, 
      hidden_size1=196, 
      hidden_size2=128, 
      latent_size=15)

In [6]:
net25b2 = models.BernoulliVAE2(input_size=204, 
      hidden_size1=196, 
      hidden_size2=128, 
      latent_size=25)

In [7]:
net50b02 = models.BernoulliVAE2(input_size=204, 
      hidden_size1=196, 
      hidden_size2=128, 
      latent_size=50)

In [7]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [9]:
def train_AE_model(net, dataset, k, epochs, batch_size):
  #https://github.com/christianversloot/machine-learning-articles/blob/main/
  #how-to-use-k-fold-cross-validation-with-pytorch.md

  if __name__ == '__main__':
    
    # Configuration options
    k_folds = k
    epochs = epochs
    #loss_function = nn.CrossEntropyLoss()
    loss_function = nn.MSELoss()
    
    # For fold results
    train_results = {}
    val_results = {}
    
    # Set fixed random number seed
    torch.manual_seed(42)
    
    # Prepare dataset by concatenating Train/Test part; we split later.

    dataset = dataset
    
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True)
      
    # Start print
    print('--------------------------------')

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
      
      # Print
      print(f'FOLD {fold}')
      print('--------------------------------')
      
      # Sample elements randomly from a given list of ids, no replacement.
      train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
      test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
      
      # Define data loaders for training and testing data in this fold
      trainloader = torch.utils.data.DataLoader(
                        dataset, 
                        batch_size=batch_size, sampler=train_subsampler)
      testloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=test_subsampler)
      

      net = net
      net.apply(reset_weights)

      # create an optimizer object
      # Adam optimizer with learning rate 1e-3
      optimizer = torch.optim.Adam(net.parameters(), lr=0.001)



      # Run the training loop for defined number of epochs
      for epoch in range(1, epochs + 1):
          print(f'\n Epoch {epoch}')
          loss = 0
          training_loss = 0
          test_loss = 0
          val_latents = []
          val_outputs = []
          train_outputs = []
          for i, (batch_features) in enumerate(trainloader):
              optimizer.zero_grad()
              # Reshape data so each image is an array with 784 elements
              batch_features = batch_features.view(-1, 204)

              output, latent = net(batch_features)
              criterion = loss_function(output, batch_features)
              criterion.backward()
              optimizer.step()

              #avg_loss += loss.item()/len(train_data)

              #z, mu_e, logvar_e = net.encode(batch_features)
              #output = net.decode(z)
              
              train_outputs.append(output.detach().numpy())
              
              train_loss = loss_function(output, batch_features)
              
              # Print statistics
              loss += train_loss.item()
              training_loss += train_loss.item()
              
              

              if i % 100 == 0:
                  # Print average loss per sample in batch
                  batch_loss = loss/len(batch_features)
                  print(f'\r[{i:d}/{len(batch_features):d}] batch loss: {batch_loss} ',
                        end='', flush=True)
              loss = 0
            

      # Process is complete.
      print('Training process has finished. Saving trained model.')

      # Saving the model
      save_path = f'/content/drive/MyDrive/Data/AE50_batch{batch_size}_fold{fold}'
      torch.save(net.state_dict(), save_path)

      # Print about testing
      #print('Starting testing')

      # Evaluation for this fold

      with torch.no_grad():

        # Iterate over the test data and generate predictions
        for i, (batch_features) in enumerate(testloader):

            # Reshape data so each image is an array with 784 elements
            batch_features = batch_features.view(-1, 204)

            #test_loss = net(batch_features)
            output, latent = net(batch_features)
            
            val_outputs.append(output.detach().numpy())
            val_latents.append(latent.detach().numpy())
            val_loss = loss_function(output, batch_features)




            test_loss += val_loss.item()


        train_outputs = np.concatenate( train_outputs, axis=0 )
        val_outputs = np.concatenate( val_outputs, axis=0 )
        val_latents = np.concatenate( val_latents, axis=0 )
        model_train_loss = training_loss / len(train_outputs
                          )
        test_loss = test_loss / len(val_outputs
                          )
        #animator.add(epoch, (loss, test_loss))
        #print(train_iter[0][:5])
        #print(outputs[:5])




        # Print fold loss
        print(f'Training loss for fold {fold}: {model_train_loss}')
        print(f'Validation loss for fold {fold}: {test_loss}')
        print('--------------------------------')
        train_results[fold] = model_train_loss
        val_results[fold] = test_loss
    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    train_sum = 0.0
    val_sum = 0.0
    for key, value in train_results.items():
      print(f'Training Fold {key} Loss: {value}')
      train_sum += value
    print(f'Trainig Average Loss: {train_sum/len(train_results.items())}')
    train_avg_loss = train_sum/len(train_results.items())
    for key, value in val_results.items():
      print(f'Validation Fold {key} Loss: {value} ')
      val_sum += value
    print(f'Vaidation Average Loss: {val_sum/len(val_results.items())}')
    val_avg_loss = val_sum/len(val_results.items())

 
    full_outputs = []
    full_outputs.append(train_avg_loss)
    full_outputs.append(val_avg_loss)
    full_outputs.append(val_latents)
    full_outputs.append(val_outputs)
    full_outputs.append(train_outputs)

    

    return full_outputs

In [10]:
batch_size = 8192
epochs = 30
k = 5

In [11]:
full_outputs_50 = train_AE_model(net=net50,
                                 dataset=dataset,
                                 k=k, 
                                 epochs=epochs,
                                 batch_size=batch_size)

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Linear(in_features=204, out_features=196, bias=True)
Reset trainable parameters of layer = Linear(in_features=196, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=196, bias=True)
Reset trainable parameters of layer = Linear(in_features=196, out_features=204, bias=True)

 Epoch 1
[0/8192] batch loss: 3.067620127694681e-05 
 Epoch 2
[0/8192] batch loss: 2.9660883228643797e-05 
 Epoch 3
[0/8192] batch loss: 2.7814106942969374e-05 
 Epoch 4
[0/8192] batch loss: 2.3081167455529794e-05 
 Epoch 5
[0/8192] batch loss: 1.3754688552580774e-05 
 Epoch 6
[0/8192] batch loss: 8.153146154654678e-06 
 Epoch 7
[0/8192] batch loss: 7.682006071263459e-06 
 Epoch 

In [8]:
def train_VAE_model(net, dataset, k, epochs, batch_size, beta, latent_size):
  #https://github.com/christianversloot/machine-learning-articles/blob/main/
  #how-to-use-k-fold-cross-validation-with-pytorch.md

  if __name__ == '__main__':
    
    # Configuration options
    k_folds = k
    epochs = epochs
    #loss_function = nn.CrossEntropyLoss()
    loss_function = nn.MSELoss()
    
    # For fold results
    train_results = {}
    val_results = {}
    
    # Set fixed random number seed
    torch.manual_seed(42)
    
    # Prepare dataset by concatenating Train/Test part; we split later.

    dataset = dataset
    
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True)
      
    # Start print
    print('--------------------------------')

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
      
      # Print
      print(f'FOLD {fold}')
      print('--------------------------------')
      
      # Sample elements randomly from a given list of ids, no replacement.
      train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
      test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
      
      # Define data loaders for training and testing data in this fold
      trainloader = torch.utils.data.DataLoader(
                        dataset, 
                        batch_size=batch_size, sampler=train_subsampler)
      testloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=test_subsampler)
      

      net = net
      net.apply(reset_weights)

      # create an optimizer object
      # Adam optimizer with learning rate 1e-3
      optimizer = torch.optim.Adam(net.parameters(), lr=0.001)



      # Run the training loop for defined number of epochs
      for epoch in range(1, epochs + 1):
          print(f'\n Epoch {epoch}')
          loss = 0
          training_loss = 0
          test_loss = 0
          val_latents = []
          val_outputs = []
          train_outputs = []
          for i, (batch_features) in enumerate(trainloader):
              optimizer.zero_grad()
              # Reshape data so each image is an array with 784 elements
              batch_features = batch_features.view(-1, 204)

              criterion, output, latent = net(batch_features, beta)
              criterion.backward()
              optimizer.step()

              #avg_loss += loss.item()/len(train_data)

              #z, mu_e, logvar_e = net.encode(batch_features)
              #output = net.decode(z)
              
              train_outputs.append(output.detach().numpy())
              
              train_loss = loss_function(output, batch_features)
              
              # Print statistics
              loss += train_loss.item()
              training_loss += train_loss.item()
              
              

              if i % 100 == 0:
                  # Print average loss per sample in batch
                  batch_loss = loss/len(batch_features)
                  print(f'\r[{i:d}/{len(batch_features):d}] batch loss: {batch_loss} ',
                        end='', flush=True)
              loss = 0
            

      # Process is complete.
      print('Training process has finished. Saving trained model.')
      # Saving the model
      save_path = f'/content/drive/MyDrive/Data/VAE{latent_size}_beta{beta}_batch{batch_size}_fold{fold}'
      torch.save(net.state_dict(), save_path)

      # Print about testing
      #print('Starting testing')

      # Evaluation for this fold

      with torch.no_grad():

        # Iterate over the test data and generate predictions
        for i, (batch_features) in enumerate(testloader):

            # Reshape data so each image is an array with 784 elements
            batch_features = batch_features.view(-1, 204)

            #test_loss = net(batch_features)
            criterion, output, latent = net(batch_features, beta)
            
            val_outputs.append(output.detach().numpy())
            val_latents.append(latent.detach().numpy())
            val_loss = loss_function(output, batch_features)




            test_loss += val_loss.item()


        train_outputs = np.concatenate( train_outputs, axis=0 )
        val_outputs = np.concatenate( val_outputs, axis=0 )
        val_latents = np.concatenate( val_latents, axis=0 )
        model_train_loss = training_loss / len(train_outputs
                          )
        test_loss = test_loss / len(val_outputs
                          )
        #animator.add(epoch, (loss, test_loss))
        #print(train_iter[0][:5])
        #print(outputs[:5])




        # Print fold loss
        print(f'Training loss for fold {fold}: {model_train_loss}')
        print(f'Validation loss for fold {fold}: {test_loss}')
        print('--------------------------------')
        train_results[fold] = model_train_loss
        val_results[fold] = test_loss
    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    train_sum = 0.0
    val_sum = 0.0
    for key, value in train_results.items():
      print(f'Training Fold {key} Loss: {value}')
      train_sum += value
    print(f'Trainig Average Loss: {train_sum/len(train_results.items())}')
    train_avg_loss = train_sum/len(train_results.items())
    for key, value in val_results.items():
      print(f'Validation Fold {key} Loss: {value} ')
      val_sum += value
    print(f'Vaidation Average Loss: {val_sum/len(val_results.items())}')
    val_avg_loss = val_sum/len(val_results.items())

    full_outputs = []
    full_outputs.append(train_avg_loss)
    full_outputs.append(val_avg_loss)
    full_outputs.append(val_latents)
    full_outputs.append(val_outputs)
    full_outputs.append(train_outputs)

    

    return full_outputs

        

In [9]:
batch_size = 128
epochs = 50
k = 5


In [14]:
full_outputs_15b2 = train_VAE_model(net=net15b2,
                                 dataset=dataset,
                                 k=k, 
                                 epochs=epochs,
                                 batch_size=batch_size,
                                 beta = 0.2,
                                 latent_size = 15)

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Linear(in_features=204, out_features=196, bias=True)
Reset trainable parameters of layer = Linear(in_features=196, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=15, bias=True)
Reset trainable parameters of layer = Linear(in_features=15, out_features=15, bias=True)
Reset trainable parameters of layer = Linear(in_features=15, out_features=15, bias=True)
Reset trainable parameters of layer = Linear(in_features=15, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=196, bias=True)
Reset trainable parameters of layer = Linear(in_features=196, out_features=204, bias=True)

 Epoch 1
[200/128] batch loss: 0.0003535336581990123 
 Epoch 2
[200/128] batch loss: 0.0003474996774457395 
 Epoch 3
[200/128] batch loss: 0.000325811211951077 
 Epoch 4
[200/128] batch loss: 0.0002950389

KeyboardInterrupt: ignored

In [10]:
full_outputs_25b2 = train_VAE_model(net=net25b2,
                                 dataset=dataset,
                                 k=k, 
                                 epochs=epochs,
                                 batch_size=batch_size,
                                 beta = 0.2,
                                 latent_size = 25)

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Linear(in_features=204, out_features=196, bias=True)
Reset trainable parameters of layer = Linear(in_features=196, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=25, bias=True)
Reset trainable parameters of layer = Linear(in_features=25, out_features=25, bias=True)
Reset trainable parameters of layer = Linear(in_features=25, out_features=25, bias=True)
Reset trainable parameters of layer = Linear(in_features=25, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=196, bias=True)
Reset trainable parameters of layer = Linear(in_features=196, out_features=204, bias=True)

 Epoch 1
[200/128] batch loss: 0.00040188070852309465 
 Epoch 2
[200/128] batch loss: 0.00036357383942231536 
 Epoch 3
[200/128] batch loss: 0.0003739433886948973 
 Epoch 4
[200/128] batch loss: 0.0003232

In [ ]:
full_outputs_15b1 = train_VAE_model(net=net15b2,
                                 dataset=dataset,
                                 k=k, 
                                 epochs=epochs,
                                 batch_size=batch_size,
                                 beta = 1,
                                 latent_size = 15)

In [15]:
full_outputs_15b5 = train_VAE_model(net=net15b2,
                                 dataset=dataset,
                                 k=k, 
                                 epochs=epochs,
                                 batch_size=batch_size,
                                 beta = 0.5,
                                 latent_size = 15)

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Linear(in_features=204, out_features=196, bias=True)
Reset trainable parameters of layer = Linear(in_features=196, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=15, bias=True)
Reset trainable parameters of layer = Linear(in_features=15, out_features=15, bias=True)
Reset trainable parameters of layer = Linear(in_features=15, out_features=15, bias=True)
Reset trainable parameters of layer = Linear(in_features=15, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=196, bias=True)
Reset trainable parameters of layer = Linear(in_features=196, out_features=204, bias=True)

 Epoch 1
[200/128] batch loss: 0.00038746255449950695 
 Epoch 2
[200/128] batch loss: 0.0003305884310975671 
 Epoch 3
[200/128] batch loss: 0.00035569979809224606 
 Epoch 4
[200/128] batch loss: 0.0003674

In [ ]:
full_outputs_50_1 = train_VAE_model(net=net50b02,
                                 dataset=dataset,
                                 k=k, 
                                 epochs=epochs,
                                 batch_size=batch_size,
                                 beta = 0.02,
                                 latent_size = 50)

In [ ]:
full_outputs_50_1 = train_VAE_model(net=net50b02,
                                 dataset=dataset,
                                 k=k, 
                                 epochs=epochs,
                                 batch_size=batch_size,
                                 beta = 0.5,
                                 latent_size = 50)